In [1]:
import time
import pandas as pd
from datetime import datetime
import firebase_admin
from firebase_admin import credentials, firestore

In [2]:
cred = credentials.Certificate('../../src/config/argoai-63051-firebase-adminsdk-ykwbi-a14bbb8c68.json')

app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [3]:
bulk_writer = db.bulk_writer()
batch = db.batch()
collection = db.collection('mrv_system')

In [4]:
location_data = ['latitude', 'longitude', 'district', 'province', 'address']
weather_data = ['humidity', 'rainfall', 'atmospheric_pressure',
                'max_temperature', 'cloud', 'wind_direction',
                'min_temperature', 'wind']
pest_data = ['pest_population_counts', 'disease_incidence', 'severity_of_infestations']
soil_data = ['soil_nutrient_levels', 'soil_moisture', 'soil_temperature', 'soil_ph']
irrigation_data = ['water_discharged', 'water_quality', 'water_consumed', 'water_withdrawn', 'water_recycled']


In [5]:
def create_data_item(row):
    return {
        'fertilize': row.get('fertilize', None),
        'weather': {key: row.get(key, None) for key in weather_data},
        'pest': {key: row.get(key, None) for key in pest_data},
        'irrigation': {key: row.get(key, None) for key in irrigation_data},
        'soil': {key: row.get(key, None) for key in soil_data},
    }

In [6]:
def write_data_to_firestore(df):
    bulk_writer = db.bulk_writer()

    for index, row in df.iterrows():
        print(f"index: {index}")
        data_item = create_data_item(row)
        facility = row.get('facility', None)
        doc_ref = db.collection("mrv_system").document()
        data = {
            # 'date': datetime.combine(row['datetime'], datetime.min.time()),
            'plant': str(row['plant']).lower().strip(),
            'date': "32/05/2024",
            # 'date': row['datetime'].strftime("%d/%m/%Y"),
            'facility': str(facility).lower(),
            'location': {key: row.get(key, None) for key in location_data},
            'data': data_item
        }
        print(f"data: {data}")
        print("________")
        bulk_writer.set(doc_ref, data)

    bulk_writer.flush()

In [7]:
df = pd.read_excel('../data/data.xlsx', sheet_name='data_temp')

In [8]:
df = df[:2]

In [9]:

start = time.time()
write_data_to_firestore(df)
end = time.time()
print(f'Execution time: {end - start} seconds')

index: 0
data: {'plant': 'rice', 'date': '32/05/2024', 'facility': 'farm22z', 'location': {'latitude': 11.8819075448256, 'longitude': 108.561573082241, 'district': 'Da Lat', 'province': 'Lam Dong', 'address': 'VHJ6+PH3, Đất, Thôn, Cầu, Xuân Trường, Thành phố Đà Lạt, Lâm Đồng, Vietnam'}, 'data': {'fertilize': None, 'weather': {'humidity': 65.16, 'rainfall': None, 'atmospheric_pressure': 1002.02, 'max_temperature': 27.49, 'cloud': 77.71, 'wind_direction': 'N', 'min_temperature': 10.47, 'wind': 9.63}, 'pest': {'pest_population_counts': 48, 'disease_incidence': 4.52, 'severity_of_infestations': 1.22}, 'irrigation': {'water_discharged': None, 'water_quality': 1.3, 'water_consumed': 887.23, 'water_withdrawn': None, 'water_recycled': 368.76}, 'soil': {'soil_nutrient_levels': None, 'soil_moisture': 14.35, 'soil_temperature': 26.69, 'soil_ph': None}}}
________
index: 1
data: {'plant': 'rice', 'date': '32/05/2024', 'facility': 'farm22z', 'location': {'latitude': 11.8819075448256, 'longitude': 10